# Notebook setup

## Install required packages

In [ ]:
!pip install -r requirements.txt --pre

## Setup logging

In [ ]:
import logging
import sys

def configure_logging(level="DEBUG"):
    """This function configures logging for code being run based on the specified level.
    Args:
        level (str): The logging level to use (e.g., "DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL").
    """

    try:
        # Convert the level string to uppercase so it matches what the logging library expects
        logging_level = getattr(logging, level.upper(), None)

        # Setup a logging format
        logging.basicConfig(
            level=logging_level,
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
            handlers=[logging.StreamHandler(sys.stdout)]
        )
    except Exception as e:
        print(f"Failed to set up logging: {e}", file=sys.stderr)
        sys.exit(1) 

configure_logging("ERROR")



## (OPTIONAL) Setup support for a proxy for inspection of calls

In [ ]:
import os

# Setup proxy through local proxy; bypass proxy for metadata endpoint to avoid any issues with Microsoft authentication libraries
os.environ["HTTP_PROXY"] = "http://127.0.0.1:8000"
os.environ["HTTPS_PROXY"] = "http://127.0.0.1:8000"
os.environ["NO_PROXY"] = "169.254.169.254"

# Add private CA root cert to trusted cert bundle for SDKs that use requests and httpx
os.environ["REQUESTS_CA_BUNDLE"] = "/Users/someuser/Library/Preferences/httptoolkit/ca.pem"
os.environ["SSL_CERT_FILE"] = "/Users/someuser/Library/Preferences/httptoolkit/ca.pem"

# V1 Agent Testing

## Perform an inference using the responses API

This will validate that the models are available.

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

load_dotenv(".env", override=True)

project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

with project_client.get_openai_client() as openai_client:
    response = openai_client.responses.create(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        input="What is the size of France in square miles?",
    )
    print(f"Response output: {response.output_text}")

    response = openai_client.responses.create(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        input="And what is the capital city?",
        previous_response_id=response.id,
    )
    print(f"Response output: {response.output_text}")

## Create, run, and delete a basic v1 agent

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder

load_dotenv(".env", override=True)

agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

with agents_client:
    # Create agent
    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="writing-agent",
        instructions="You are a helpful writing assistant"
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create thread
    thread = agents_client.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    # Create message
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="Write me a poem about flowers"
    )
    print(f"Created message, message ID: {message.id}")

    # Run agent and process
    run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    else:
        # Get the last message from the agent
        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"{msg.role}: {last_text.text.value}")

    # Cleanup
    #agents_client.delete_agent(agent.id)
    #print("Deleted agent")

## Create and run a v1 agent that uses the file analysis tool

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder, FileSearchTool
from openai import project

load_dotenv(".env", override=True)

agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Upload file and create vector store
file = agents_client.files.upload(file_path="./sample-files/product_info_1.md", purpose=FilePurpose.AGENTS)
vector_store = agents_client.vector_stores.create_and_poll(file_ids=[file.id], name="my_vectorstore")

# Create file search tool and agent
# This tool will upload the file to the storage account then create an index for it in the AI Search instance
file_search = FileSearchTool(vector_store_ids=[vector_store.id])
agent = agents_client.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],
    name="file-agent",
    instructions="You are a helpful assistant and can search information from uploaded files",
    tools=file_search.definitions,
    tool_resources=file_search.resources,
)

# Create thread and process user message
thread = agents_client.threads.create()
agents_client.messages.create(thread_id=thread.id, role="user", content="Hello, what Contoso products do you know?")
run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)

# Handle run status
if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Print thread messages
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
for message in messages:
    if message.run_id == run.id and message.text_messages:
        print(f"{message.role}: {message.text_messages[-1].text.value}")

# Cleanup resources
agents_client.vector_stores.delete(vector_store.id)
agents_client.files.delete(file_id=file.id)
agents_client.delete_agent(agent.id)

## Create and run an agent that uses a public MCP Tool

### Find the tools advertised by the MCP Server

Use the FastMCP library to get a list of tools available behind the MCP Server

In [ ]:
import asyncio
from fastmcp import Client

async def list_mcp_tools(server_url: str):
    """List tools available from an MCP server."""
    # Setup an async client to connect to the MCP server
    async with Client(server_url) as client:
        tools = await client.list_tools()
        
        # Write out each tool returned
        print(f"Found {len(tools)} tools:")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")

await list_mcp_tools("https://learn.microsoft.com/api/mcp")

### Run the agent and force call to the public MCP Server

In [ ]:
import os
import time
from dotenv import load_dotenv
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    ListSortOrder, 
    McpTool, 
    MCPToolResource,
    ToolResources
)
from fastmcp.tools import tool

load_dotenv(".env", override=True)

# Create MCP tool for the Microsoft Learn MCP Server
mcp_tool = McpTool(
    server_label="ms_learn",
    server_url="https://learn.microsoft.com/api/mcp",
    allowed_tools=["microsoft_docs_search"]
)

# Setup an AgentsClient
agents_client = AgentsClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

with agents_client:
    # Create agent with MCP tool
    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="mcp-agent",
        instructions="You are a helpful assistant who answers questions about Microsoft products using Microsoft Learn content. Use the available MCP tools to answer questions.",
        tools=mcp_tool.definitions,
        tool_resources=mcp_tool.resources
    )

    # Create thread
    thread = agents_client.threads.create()
    print(f"Created thread, thread ID: {thread.id}")

    # Create message
    message = agents_client.messages.create(
        thread_id=thread.id,
        role="user",
        content="What is the largest SKU for an ExpressRoute Gateway?"
    )
    print(f"Created message, message ID: {message.id}")

    # Create run and poll for completion, handling tool approvals
    run = agents_client.runs.create(
        thread_id=thread.id, 
        agent_id=agent.id,
        # Set the tool calls to be automatically approved
        tool_resources=ToolResources(
            mcp=[
                MCPToolResource(
                    server_label="ms_learn",
                    headers={},
                    require_approval="never"
                )
            ]
        )
    )
    print(f"Created run, ID: {run.id}")

    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

        print(f"Run status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    else:
        # Get messages
        messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        print("\nConversation:")
        print("-" * 50)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"{msg.role}: {last_text.text.value}")
                print("-" * 50)
    
    print(f"Run completed with status: {run.status}")
    # Cleanup
    agents_client.delete_agent(agent.id)
    print("Deleted agent")

# V2 Agent Testing

## Create, run, and delete a basic v2 agent
This will validate that v2 agents can be created. This helps to validate a good chunk of the setup, including connectivity with the CosmosDB

In [ ]:
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import PromptAgentDefinition

load_dotenv(".env", override=True)

project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

with project_client.get_openai_client() as openai_client:
    agent = project_client.agents.create_version(
        agent_name="basic-v2-agent",
        definition=PromptAgentDefinition(
            model=os.environ["MODEL_DEPLOYMENT_NAME"],
            instructions="You are a helpful assistant that answers general questions",
        ),
    )
    print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

    conversation = openai_client.conversations.create(
        items=[{"type": "message", "role": "user", "content": "What is the size of France in square miles?"}],
    )
    print(f"Created conversation with initial user message (id: {conversation.id})")

    response = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        input="",
    )
    print(f"Response output: {response.output_text}")

    openai_client.conversations.items.create(
        conversation_id=conversation.id,
        items=[{"type": "message", "role": "user", "content": "And what is the capital city?"}],
    )
    print(f"Added a second user message to the conversation")

    response = openai_client.responses.create(
        conversation=conversation.id,
        extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
        input="",
    )
    print(f"Response output: {response.output_text}")

    openai_client.conversations.delete(conversation_id=conversation.id)
    print("Conversation deleted")

project_client.agents.delete_version(agent_name=agent.name, agent_version=agent.version)
print("Agent deleted")